<a href="https://colab.research.google.com/github/xbarusui/study_gpt2-japanese_V2/blob/main/study_gpt2_japaneseV2_AuthorAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **作家AI の文章学習・生成**

メニュー

*   1.前準備(googleドライブ)
*   2.前準備(モジュールインストール)
*   3.作家AI学習（学習不要な場合はスキップできます）
*   4.作家AI文章生成

利用方法

*   作家AIを作り文章生成させる場合は、<font color= "blue">**1-4 の順に**</font>実行してください
*   とりあえず文章生成を試したい場合は、<font color= "blue"> **1,3,4 の順に**</font> 実行して、Default で文章を生成してください
*   学習させると作家AI はgoogleドライブに1-10の番号で格納されます
*   作成済みの作家AIを利用する場合も、 <font color= "blue">**1,3,4 の順に**</font> 実行してください


# **<font color= "red">1.前準備(googleドライブ)</font>**


In [ ]:
#@title googleドライブに接続して gpt2work というフォルダを作成

#@markdown **<font color= "blue">左のセルを実行してください</font>**

from google.colab import drive 
drive.mount('/content/drive')
!mkdir -p '/content/drive/My Drive/gpt2work/'
%cd '/content/drive/My Drive/gpt2work/'

## 上記の説明

1．「Googleドライブに接続してgpt2workというフォルダを作成」というセルを実行すると、URLと入力欄が表示されます。

![Googleドライブ認証](https://github.com/xbarusui/study_gpt2-japanese_V2/blob/main/authimage1.png?raw=true)

2．青いURLをクリックすると、新しいタブが開いて、colabと連携するアカウントを聞かれます（聞かれなかった場合、次に進んで下さい）

![アカウントの選択](https://github.com/xbarusui/study_gpt2-japanese_V2/blob/main/authimage2.png?raw=true)

3．「このアプリをGoogleからダウンロードしたことをご確認ください」と出ますので確認の上、「ログイン」を選択してください。

![代替テキスト](https://github.com/xbarusui/study_gpt2-japanese_V2/blob/main/authimage3.png?raw=true)

4．許可すると、認証コードの横にコピーボタンが表示されますので、コピーボタンをクリックして下さい。

![認証コード](https://github.com/xbarusui/study_gpt2-japanese_V2/blob/main/authimage4.png?raw=true)

5．colabの画面に戻ってきて、URLの下の入力欄にカーソルを合わせて、「Ctrl+V」で貼り付けて下さい。文字列は・で表示されます。

![認証コード貼り付け](https://github.com/xbarusui/study_gpt2-japanese_V2/blob/main/authimage5.png?raw=true)




# **<font color= "red">2.前準備(モジュールインストール)</font>**

In [ ]:
#@title 学習済みモデルと周辺モジュールインストール

#@markdown **<font color= "blue">左のセルを実行してください</font>**


# ディレクトリ移動
%cd '/content/drive/My Drive/gpt2work/'

!git clone https://github.com/rinnakk/japanese-pretrained-models
%cd japanese-pretrained-models
!pip install -r requirements.txt

# Huggingface Datasetsのインストール
!pip install datasets==1.2.1

# Sentencepieceのインストール
!pip install sentencepiece==0.1.91

# 初めての場合は、gpt2work以下に下のフォルダがないか確認し、無ければモジュールをgithubダウンロード
import os
path = 'study_gpt2-japanese_V2'
is_dir = os.path.isdir(path)

if is_dir is False:
  !git clone https://github.com/xbarusui/study_gpt2-japanese_V2


# **<font color= "red">3.作家AI学習（学習不要な場合はスキップできます）</font>**

In [ ]:
#@title 学習させたいテキストファイルをUPLOADする
#@markdown **<font color= "blue">左のセルを実行して、出てくるウィンドウで学習させたい文章のテキストをアップロードしてください</font>**

#@markdown 注1.テキストはUTF-8形式でないと読み込めません

#@markdown 注2.既に同名テキストが格納してある場合、ファイル名が(1)のように連番になって上書きせずにアップロードされますので、ご注意ください


%cd '/content/drive/My Drive/gpt2work/'

from google.colab import files
uploaded = files.upload()

!ls -l | grep .txt


In [ ]:
#@title パラメータ設定後に学習開始
#@markdown **<font color= "blue">下にある3パラメータを設定してから、左のセルを実行してください</font>**

%cd '/content/drive/My Drive/gpt2work/'

#@markdown **<font color= "blue"> 1.学習結果を格納するフォルダをドロップダウンから選択してください</font>**

#@markdown 選択した番号のフォルダに作家AIが格納されます

authormodel = "1" #@param ["1","2","3","4","5","6","7","8","9","10"]

#@markdown 

savefolder = authormodel + "/"

#@markdown **<font color= "blue"> 2.アップロードしたテキスト名を入力してください</font>**

#@markdown 同名ファイルをアップロードしている場合、名称が連番になるので最新のテキスト名を指定してください

upload_file = ""  #@param {type:"string"}

#@markdown

#@markdown **<font color= "blue"> 3.何エポック学習するかを数字で設定してください</font>**

#@markdown エポックというのは訓練データを学習する単位になります。初期値は 30 ですが、それ以上学習させると精度が上がります。

#@markdown 簡単に利用するだけなら 30-300 の範囲で1時間程度学習させるくらいで十分ではないかと思います。

num_train_epochs = 30  #@param {type:"number"}


# ファインチューニングの実行
!python study_gpt2-japanese_V2/gpt2japanese_study.py \
    --model_name_or_path=rinna/japanese-gpt2-xsmall \
    --train_file={upload_file} \
    --validation_file={upload_file} \
    --do_train \
    --do_eval \
    --num_train_epochs={num_train_epochs} \
    --save_steps=5000 \
    --save_total_limit=3 \
    --per_device_train_batch_size=1 \
    --per_device_eval_batch_size=1 \
    --output_dir={savefolder} \
    --use_fast_tokenizer=False \
    --overwrite_output_dir

# **<font color= "red">4.作家AI文章生成</font>**

In [ ]:
%cd '/content/drive/My Drive/gpt2work/'
#@title パラメータ設定後に文章生成
#@markdown **<font color= "blue">下にある4パラメータを設定してから、左のセルを実行してください</font>**


#@markdown **<font color= "blue"> 1.作家AIのモデルをドロップダウンから選択してください</font>**

#@markdown Default を選択した場合は初期の学習済みモデルを利用し、数字を選択した場合は上で学習した番号に格納された作家AIを利用します

authormodel = "Default" #@param ["Default", "1","2","3","4","5","6","7","8","9","10"]

#@markdown 

from transformers import T5Tokenizer, AutoModelForCausalLM

if authormodel == "Default" :
  # ライブラリをインポートします。
  # 学習済みモデルをダウンロードします。
  savefolder = "rinna/japanese-gpt2-xsmall"
else:
  savefolder = authormodel + "/"

#@markdown **<font color= "blue"> 2.書かせる文章の最初の部分を記載してください</font>**

#@markdown 最初の文章が長い方が出力される文章は長くなる傾向にあります

noveltext = "\u3053\u3093\u306B\u3061\u306F"  #@param {type:"string"}

#@markdown 

#@markdown **<font color= "blue"> 3.出力文字数を指定してください</font>**

#@markdown モデルや入力された文章によっては出力文字数は半分くらいに短くなることもあります

novellength = 128  #@param {type:"number"}

#@markdown 

#@markdown **<font color= "blue"> 4.何回生成するかを選択してください</font>**

#@markdown 回数が多いほど生成時間は長くなります

novelseq =   5#@param {type:"number"}


# トークナイザーとモデルの準備
tokenizer = T5Tokenizer.from_pretrained("rinna/japanese-gpt2-xsmall")
model = AutoModelForCausalLM.from_pretrained(savefolder)

# 生成された文章を表示します。
print("冒頭の文章")
print("----------------------------------------")
print(noveltext)
print("----------------------------------------")

# 推論
input = tokenizer.encode( noveltext , return_tensors="pt")
output = model.generate(input, do_sample=True, max_length=novellength, num_return_sequences=novelseq,top_k=0,min_length=int(novellength/2))
# 出力されたコードを文章に戻します。
DecodedOutput = tokenizer.batch_decode(output)

for j in range(novelseq):
  print("--- AI生成文章" + str(j+1) + "回目 ---")
  i = 0
  while i < len(DecodedOutput[j]):
    print(DecodedOutput[j].replace('</s>','')[i:i+80]) 
    i = i+80
  print("----------------------------------------")

  